In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
data=pd.read_csv(r"C:\Users\User\Desktop\Machine Learning Project\Biodiesel-Yield-Using-ANN-main\xanghum_data.csv")
data

,X1,X2,X3,Observed
0,1,2,7.5,7.22
1,3,2,7.5,7.52
2,1,6,7.5,4.58
3,3,6,7.5,6.45
4,1,4,0.0,4.47
5,3,4,0.0,5.67
6,1,4,15.0,6.46
7,3,4,15.0,7.76
8,2,2,0.0,6.65
9,2,6,0.0,4.52


In [3]:
def get_preds(y_test, y_preds):
    y_test=pd.DataFrame(y_test)
    y_test.rename(columns={0:'Actual'}, inplace=True)
    y_preds=pd.DataFrame(y_preds)
    y_preds.rename(columns={0:'Predicted'}, inplace=True)
    predictions=pd.concat([y_test, y_preds], axis=1)
    return predictions    

In [4]:
from sklearn.model_selection import train_test_split, cross_val_score,KFold, GridSearchCV
from sklearn.svm import SVR
from sklearn.preprocessing import StandardScaler
from math import sqrt
from sklearn.metrics import r2_score, mean_absolute_error,mean_squared_error

In [5]:
target="Observed"
X=data.drop(columns=[target]).values
y=data[target].values.reshape(-1,1)
X_train, X_test, y_train, y_test= train_test_split(X, y, test_size=0.3, random_state=100)

In [6]:
from xgboost import XGBRegressor
import numpy as np

In [7]:
xgb=XGBRegressor(random_state=101)
param_grid = {
        'max_depth': range(50,100,10),
        'gamma': [0.0001,0.001,0.01],
        'subsample': np.arange(0.5,1,0.1),
        'colsample_bylevel':[0.6,0.7,0.8,0.9],
    "colsample_bylevel" : [0.6,0.7,0.8,0.9],
    'n_estimators' : range(100,1100,100),
}

In [8]:
gsearch = GridSearchCV(estimator = xgb,
                           param_grid = param_grid,                        
                           cv =3,
                           n_jobs = -1,
                           verbose = True)
%time gsearch.fit(X_train, y_train)

Fitting 3 folds for each of 3000 candidates, totalling 9000 fits
Wall time: 21min 47s


GridSearchCV(cv=3,
             estimator=XGBRegressor(base_score=None, booster=None,
                                    callbacks=None, colsample_bylevel=None,
                                    colsample_bynode=None,
                                    colsample_bytree=None,
                                    early_stopping_rounds=None,
                                    enable_categorical=False, eval_metric=None,
                                    gamma=None, gpu_id=None, grow_policy=None,
                                    importance_type=None,
                                    interaction_constraints=None,
                                    learning_rate=None, max_bin=None,
                                    max_cat...
                                    monotone_constraints=None, n_estimators=100,
                                    n_jobs=None, num_parallel_tree=None,
                                    predictor=None, random_state=101,
                                    reg_alpha=None, reg_lambda=None, ...),
             n_jobs=-1,
             param_grid={'colsample_bylevel': [0.6, 0.7, 0.8, 0.9],
                         'gamma': [0.0001, 0.001, 0.01],
                         'max_depth': range(50, 100, 10),
                         'n_estimators': range(100, 1100, 100),
                         'subsample': array([0.5, 0.6, 0.7, 0.8, 0.9])},
             verbose=True)

In [9]:
best_xgb=gsearch.best_estimator_
best_xgb

XGBRegressor(base_score=0.5, booster='gbtree', callbacks=None,
             colsample_bylevel=0.7, colsample_bynode=1, colsample_bytree=1,
             early_stopping_rounds=None, enable_categorical=False,
             eval_metric=None, gamma=0.001, gpu_id=-1, grow_policy='depthwise',
             importance_type=None, interaction_constraints='',
             learning_rate=0.300000012, max_bin=256, max_cat_to_onehot=4,
             max_delta_step=0, max_depth=50, max_leaves=0, min_child_weight=1,
             missing=nan, monotone_constraints='()', n_estimators=200, n_jobs=0,
             num_parallel_tree=1, predictor='auto', random_state=101,
             reg_alpha=0, reg_lambda=1, ...)

In [21]:
import time
start_time=time.time()
best_xgb=gsearch.best_estimator_
best_xgb.fit(X_train,y_train)
elapsed_time=time.time()-start_time
print(f'executed in {elapsed_time} seconds or {elapsed_time*1000} milliseconds')
pred_train=best_xgb.predict(X_train)

executed in 0.5101513862609863 seconds or 510.1513862609863 milliseconds


In [22]:
xgb_prediction_train=get_preds(y_train, pred_train)
xgb_prediction_train

,Actual,Predicted
0,6.22,6.075117
1,5.67,5.659954
2,4.58,4.571248
3,4.47,4.458902
4,8.31,8.317501
5,7.22,7.196576
6,6.21,6.075117
7,7.76,7.743559
8,6.45,6.404911
9,5.86,6.075117


In [18]:
corr_train=xgb_prediction_train["Actual"].corr(xgb_prediction_train["Predicted"])
corr_train

0.9969158631431235

In [20]:
print('TRAIN root mean squared error is {}'.format(np.sqrt(mean_squared_error(y_train,pred_train))))
print('TRAIN Mean absolute error is {}'.format(mean_absolute_error(y_train,pred_train)))
print('TRAIN R2 score is {}'.format(r2_score(y_train,pred_train)))

TRAIN root mean squared error is 0.08991755041598058
TRAIN Mean absolute error is 0.05760047739202308
TRAIN R2 score is 0.993680991719319


In [273]:
pred_test=best_xgb.predict(X_test)

In [282]:
xgb_prediction_test=get_preds(y_test, pred_test)
xgb_prediction_test

,Actual,Predicted
0,6.28,6.075117
1,4.52,4.943915
2,6.93,6.594424
3,7.52,7.502203
4,5.81,6.075117
5,6.46,6.443666


In [285]:
corr_test=xgb_prediction_test["Actual"].corr(xgb_prediction_test["Predicted"])
corr_test

0.9752571503905806

In [283]:
print('TEST root mean squared error is {}'.format(np.sqrt(mean_squared_error(y_test,pred_test))))
print('TEST Mean absolute error is {}'.format(mean_absolute_error(y_test,pred_test)))
print('TEST R2 score is {}'.format(r2_score(y_test,pred_test)))

TEST root mean squared error is 0.2598598225774433
TEST Mean absolute error is 0.21060353279113775
TEST R2 score is 0.9236511920064041


In [287]:
pred_df=best_xgb.predict(X)

In [288]:
xgb_prediction_df=get_preds(y, pred_df)
xgb_prediction_df

,Actual,Predicted
0,7.22,7.196576
1,7.52,7.502203
2,4.58,4.571248
3,6.45,6.404911
4,4.47,4.458902
5,5.67,5.659954
6,6.46,6.443666
7,7.76,7.743559
8,6.65,6.666370
9,4.52,4.943915


In [289]:
corr_df=xgb_prediction_df["Actual"].corr(xgb_prediction_df["Predicted"])
corr_df

0.9878767213322027

In [291]:
print('FULL DATA root mean squared error is {}'.format(np.sqrt(mean_squared_error(y,pred_df))))
print('FULL DATA Mean absolute error is {}'.format(mean_absolute_error(y,pred_df)))
print('FULL DATA R2 score is {}'.format(r2_score(y,pred_df)))

FULL DATA root mean squared error is 0.17048366989949443
FULL DATA Mean absolute error is 0.1116015557681812
FULL DATA R2 score is 0.9745221422444582
